In [1]:
import tensorflow as tf

2024-12-09 23:24:14.939886: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-09 23:24:16.862126: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-09 23:24:17.848376: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-09 23:24:18.773473: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-09 23:24:18.960706: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-09 23:24:20.457959: I tensorflow/core/platform/cpu_feature_gu

In [2]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(32, 32, 3)),
    tf.keras.layers.Conv2D(filters=6, kernel_size=5, activation="tanh"),
    tf.keras.layers.AvgPool2D(pool_size=2),
    tf.keras.layers.Conv2D(filters=16, kernel_size=5, activation="tanh"),
    tf.keras.layers.AvgPool2D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(120, activation="tanh"),
    tf.keras.layers.Dense(84, activation="tanh"),
    tf.keras.layers.Dense(10, activation="softmax")
])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 6)      │           456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (None, 14, 14, 6)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 10, 10, 16)     │         2,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_1             │ (None, 5, 5, 16)       │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 400)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 120)            │        48,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 84)             │        10,164 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           850 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 62,006 (242.21 KB)

 Trainable params: 62,006 (242.21 KB)

 Non-trainable params: 0 (0.00 B)

![Alt text](assets/images/cnn.jpg)

**Convolution operation intuition:**
- Suppose we have an image of size 6x6x3 shown in the picture below.
- We have defined a Conv2D layer with `filters=1` and `kernel_size=(3, 3)`.
- If we would have defined the layer with `filters=2` then we would have two 3x3x3 kernels show in the picture below.
- As our image has 3 channels, we will have a kernel of size 3x3x3.
- If our image would have n channels then we would have a kernel of size 3x3xn.
- Each 3x3 grid of the kernel will slide over each chennel in the image.

![](assets/images/kernel.jpg)

**Sliding kernel over image:**

![](assets/images/sliding_kernel.jpg)
- The kernel will keep sliding over the image from top left till bottom right and keep producing a single number for each 3x3x3 region in the image.
- At the end of the convolutoin operation, we will have a single 4x4 output grid.
- If we had specified n filters then we would have n such output grids of size 4x4.

**Calculating total parameters of the model:**
- Input layer takes an image of shape 32x32x3.
- The image is then passed to Conv2D layer where kernel size is 5 and number of filters are 6 which creates 6 kernels of size 5x5x3.
- The total number of parameters in this layer are 5x5x3x6+6(1 bias term per filter) = 456.
- It generates an output of shape 28x28x6 which is then passed to the AvgPool2D layer which reduces this shape to 14x14x6.
- The output is then passed to Conv2D layer where kernel size is 5 and number of filters are 16 which creates 16 kernels of size 5x5x3.
- The total number of parameters in this layer are 5x5x6x16+16(1 bias term per filter) = 2416.
- It generates an output of shape 10x10x16 which is then passed to the AvgPool2D layer which reduces this shape to 5x5x16.
- The output is then passed to Flatten layer where the output becomes a 1d tensor of shape (400,).
- The next layer is a Dense layer having 120 units.
- Each unit will have 400 weights and 1 bias which makes a total of 401*120 = 48120 parameters in this layer.
- The next layer is a Dense layer having 84 units.
- Each unit will have 120 weights and 1 bias which makes a total of 121*84 = 10164 parameters in this layer.
- The next layer is a Dense layer having 10 units.
- Each unit will have 84 weights and 1 bias which makes a total of 85*10 = 850 parameters in this layer.
- The total number of parameters till now are 456+2416+48120+10164+850 = 62006.